In [ ]:
cols = ['description', 'h1', 'h2', 'domain']

for col in cols:
    most_frequent = train[col].value_counts().sort_values(ascending=False).index[0]
    train[col].fillna(most_frequent, inplace=True)
    test[col].fillna(most_frequent, inplace=True)
    print('All the missing values in column', col, 'are replaced with', most_frequent)
train.isna().sum()

from hazm import WordTokenizer
from hazm import stopwords_list


tokenizer = WordTokenizer()

def my_tokenizer(text):
    result = tokenizer.tokenize(text)
    return result # To-Do

def preprocessing(text):
    filtered = []
    # To-Do
    global lst
    lst = text.split(' ')
    for m in lst:
        if m not in stopwords_list():
            filtered.append(m)
    return filtered

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
train['category'] = le.fit_transform(train['category'])
to_label = dict(zip(le.classes_, le.transform(le.classes_)))
to_class = dict(zip(le.transform(le.classes_), le.classes_))
print(to_label)
train.head()
# To-Do

from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(ngram_range=(1, 2))

X_train = train.loc[:3999, 'title'].values
y_train = train.loc[:3999, 'category'].values
X_val = train.loc[4000:, 'title'].values
y_val = train.loc[4000:, 'category'].values
X_test = test.loc[:, 'title'].values

train_vectors = cv.fit_transform(X_train)
val_vectors = cv.transform(X_val)
test_vectors = cv.transform(X_test)

print(train_vectors.shape, val_vectors.shape)

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(train_vectors, y_train)

from sklearn.metrics import f1_score

predicted = clf.predict(val_vectors)
f1_score(y_val,predicted, average='weighted')

submission = pd.DataFrame()
submission['category'] = clf.predict(test_vectors)
submission.replace({0: 'اجتماعی', 1: 'اشتغال', 2: 'تجارت و اقتصاد', 3: 'تحصیلات', 4: 'تکنولوژی و کامپبوتر', 5: 'حقوق و دولت و سیاست', 6: 'حیوانات خانگی', 7: 'خانه و باغبانی', 8: 'خانواده', 9: 'خودرو', 10: 'سفر و گردشگری', 11: 'سلامت', 12: 'علم و دانش', 13: 'غذا و نوشیدنی', 14: 'فیلم و سینما', 15: 'مد و زیبایی', 16: 'مذهبی', 17: 'مسکن', 18: 'موسیقی', 19: 'هنر و سرگرمی', 20: 'ورزش', 21: 'کتاب و ادبیات'}, inplace=True)
submission
